# Strategy Report Notebook
This notebok runs analysis on a given strategy.

## Import Libraries

In [ ]:
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd
import plotly.graph_objects as go


## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for Equity Benchmark

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**):  SPTR, M1WD, SX5T

In [ ]:
equity_bmk = 'SPTR'
strat_drop_list = ['99%/90% Put Spread','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']
include_fi = False
#create returns data dictionary for equity benchmark
vortex_dict= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for a Strategy

In [ ]:
new_3_robust= pd.read_excel(dm.NEW_DATA+'Barclays_new3_robustness.xlsx',
                                           sheet_name = 'data', index_col=0)

#create returns data dictionary for strategy
new_data_dict = dm.get_data_dict(new_3_robust)

#merge dictionaries
ret_data_dict = dm.merge_dicts(vortex_dict,new_data_dict)

## View Returns Data
Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [ ]:
freq_list = ['Weekly', 'Monthly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(ret_data_dict[frequency])

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(ret_data_dict)

In [ ]:
col_dict = {'max_vix':['SPTR', 'Vortex', 'new3', 'maxVIXDelta=0.65', 'maxVIXDelta=0.7','maxVIXDelta=0.75', 
                       'maxVIXDelta=0.8', 'maxVIXDelta=0.85','maxVIXDelta=0.9', 'maxVIXDelta=1.1'],
           'short_put_strike': ['SPTR', 'Vortex', 'new3','shortPutStrikes=-0.35,-0.3,-0.25', 
                                'shortPutStrikes=-0.3,-0.25,-0.2','shortPutStrikes=-0.2', 'shortPutStrikes=-0.25',
                                'shortPutStrikes=-0.3','shortPutStrikes=-0.35', 'shortPutStrikes=-0.4'],
           'long_put_strike':['SPTR', 'Vortex', 'new3', 'longSPXStrike=-0.1','longSPXStrike=-0.15'],
           'vix_strike':['SPTR', 'Vortex', 'new3', 'VIXStrike=0.2', 'VIXStrike=0.3'],
           'roll_dates':['SPTR', 'Vortex', 'new3', 'rollDates=-20,-15,-10,-5', 'rollDates=5,10,15,20']}


In [ ]:
for key in col_dict:
    for freq in analytics_dict:
        temp_dict = {}
        temp_rs = analytics_dict[freq][False]['return_stats'].copy()
        temp_rs = temp_rs[col_dict[key]]
        temp_dict['return_stats'] = temp_rs
        temp_hm = analytics_dict[freq][False]['hedge_metrics'].copy()
        temp_hm = temp_hm[col_dict[key][1:]]
        temp_dict['hedge_metrics'] = temp_hm
        analytics_dict[freq][key] = temp_dict

## Display Return Stats and Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, param = [False]+list(col_dict.keys()), stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[param]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(ret_data_dict)
hist_list = list(hist_dict[False].columns)[0:2]
for key in col_dict:
    temp_list = hist_list + col_dict[key]
    temp_hs = hist_dict[False].copy()
    temp_hs = temp_hs[temp_list]
    hist_dict[key] = temp_hs

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs(param = [False]+list(col_dict.keys())):
    df_hist = hist_dict[param]
    return fmt.get_hist_styler(df_hist)

In [ ]:
roll_dates = pd.read_excel(dm.NEW_DATA+'Barclays_roll_plus_vix.xlsx',
                                           sheet_name = 'data', index_col=0)

#create returns data dictionary for strategy
new_data_dict_2 = dm.get_data_dict(roll_dates)

#merge dictionaries
ret_data_dict_2 = dm.merge_dicts(vortex_dict, new_data_dict_2)

In [ ]:
analytics_dict_2 = summary.get_analytics_data(ret_data_dict_2)

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_2[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

In [ ]:
hist_dict_2 = summary.get_hist_data(ret_data_dict_2)

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict_2[False]
    return fmt.get_hist_styler(df_hist)

# Create Report

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
strat_report_1 = 'new_3_robustness'
strat_report_2 = 'roll_dates_vix_payoff'
selloffs = True
rp.generate_strat_report(strat_report_1, ret_data_dict, selloffs)
rp.generate_strat_report(strat_report_2, ret_data_dict_2, selloffs)